# Day 4
https://adventofcode.com/2018/day/4

In [1]:
import aocd
data = aocd.get_data(year=2018, day=4)

In [2]:
from dataclasses import dataclass
from datetime import datetime
from typing import Dict, Set
import re

In [3]:
re_event = re.compile(r'\[([-: \d]+)\] (.+)')
re_guard_change = re.compile(r'Guard #(\d+) begins shift')
re_wakes_up = re.compile(r'wakes up')
re_falls_asleep = re.compile(r'falls asleep')

In [4]:
@dataclass(frozen=True, eq=True, order=True)
class Event():
    when: datetime
    event: str
    
    @classmethod
    def from_regex_groups(cls, groups):
        return cls(
            datetime.fromisoformat(groups[0]),
            groups[1]
        )
    
    @classmethod
    def all_from_text(cls, text):
        return sorted([cls.from_regex_groups(groups) for groups in re_event.findall(text)])

In [5]:
@dataclass(frozen=True)
class Shift():
    guard: int
    asleep: Set[int]
    
    @classmethod
    def from_events(cls, guard, events):
        awake = True
        fell_asleep = 0
        asleep_mins = set()
        for event in events:
            if re_falls_asleep.search(event.event):
                fell_asleep = event.when.minute
            if re_wakes_up.search(event.event):
                for minute in range(fell_asleep, event.when.minute):
                    asleep_mins.add(minute)
                fell_asleep = event.when.minute
        if not awake:
            for minute in range(fell_asleep, 60):
                asleep_mins.add(minute)
            
        return cls(guard, asleep_mins)
    
    @classmethod
    def all_from_events(cls, events):
        shifts = []
        guard = this_guard_start = -1
        for e in range(len(events)):
            guard_change = re_guard_change.search(events[e].event)
            if guard_change:
                if this_guard_start > -1:
                    shifts.append((guard, events[this_guard_start:e]))
                this_guard_start = e
                guard = int(guard_change.groups()[0])
        return [cls.from_events(*shift) for shift in shifts]

In [6]:
@dataclass(frozen=True)
class Guard():
    guard: int
    asleep: Dict[int, bool]
    
    @property
    def total_minutes_asleep(self):
        return sum(times for times in self.asleep.values())
    
    @property
    def minute_most_often_asleep(self):        
        return sorted([(times, minute) for (minute, times) in self.asleep.items()], reverse=True)[0][1]
    
    @property
    def most_times_asleep_on_same_minute(self):
        return sorted([(times, minute) for (minute, times) in self.asleep.items()], reverse=True)[0][0]
    
    @classmethod
    def all_from_shifts(cls, shifts):
        guards = {}
        for shift in shifts:
            if shift.guard not in guards:
                guards[shift.guard] = dict((minute, 0) for minute in range(60))
            for minute in shift.asleep:
                guards[shift.guard][minute] += 1
        
        return [cls(guard, asleep) for guard, asleep in guards.items()]

In [7]:
events = Event.all_from_text(data)
shifts = Shift.all_from_events(events)
guards = Guard.all_from_shifts(shifts)

##### Part 1: Find the guard who is most often asleep, and then the minute he most often sleeps.

In [8]:
guard = sorted(guards, key=lambda g: g.total_minutes_asleep, reverse=True)[0]
p1 = guard.guard * guard.minute_most_often_asleep
print('Part 1: {}'.format(p1))

Part 1: 151754


##### Part 2: Find the guard who is most frequently asleep on the same minute

In [9]:
guard = sorted(guards, key=lambda g: g.most_times_asleep_on_same_minute, reverse=True)[0]
p2 = guard.guard * guard.minute_most_often_asleep
print('Part 2: {}'.format(p2))

Part 2: 19896
